In [ ]:
import requests

# Base URL
QOGITA_API_URL = "https://api.qogita.com"

# Login credentials — use environment variables instead of hardcoding in real use
QOGITA_EMAIL = "jacek.budner@gmail.com"
QOGITA_PASSWORD = "JB100noga!"  # 🔒 Consider storing securely

# Login to retrieve the token and active cart ID
try:
    response = requests.post(
        url=f"{QOGITA_API_URL}/auth/login/",
        json={"email": QOGITA_EMAIL, "password": QOGITA_PASSWORD}
    )

    response.raise_for_status()
    data = response.json()

    access_token = data["accessToken"]
    cart_qid = data["user"]["activeCartQid"]

    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    print("Login successful!")
    print("Access Token:", access_token[:10] + "...")  # just to show it's retrieved
    print("Active Cart QID:", cart_qid)

except requests.exceptions.RequestException as e:
    print("Login failed:", e)
except KeyError:
    print("Unexpected response structure:", response.text)


In [3]:
print(variant)

{'gtin': '3360373063697', 'name': 'Cacharel Eau de Toilette for Women 30ml', 'slug': 'cacharel-amor-amor-30-ml-eau-de-toilette-womens-perfume', 'label': '30 Ml', 'category': {'name': 'Perfume & Cologne', 'slug': 'perfume-cologne', 'description': '', 'hscode': '33030010', 'qid': 'f480bbee-02e7-4964-846a-4d3d9a310a50'}, 'brand': {'name': 'Cacharel', 'slug': 'cacharel', 'description': '', 'qid': 'efb4ca3c-0153-4e3a-9f3a-6f702817fa96'}, 'dimensions': {'width': '0.0760', 'height': '0.0640', 'mass': 28, 'depth': '0.1020', 'qid': '415dddef-1ec5-464a-90df-b8e6aa5e8193'}, 'images': [{'url': 'https://static.prod.qogita.com/files/images/variants/3iis5LWveWCqWF56Care2N.jpg', 'alt': 'Cacharel Amor Amor Eau de Toilette for Women 30ml', 'qid': 'bb5586c2-395d-4928-b15d-a04e91883a15'}, {'url': 'https://static.prod.qogita.com/files/images/variants/cKonuvY4LiCjsEHFrQNJT8.jpg', 'alt': 'Cacharel Amor Amor Eau de Toilette for Women 30ml', 'qid': '948d79b2-c982-40d5-85fd-f16ea68a2a4c'}], 'price': None, 'pric